In [ ]:
# imports
import numpy as np
from math import factorial
import random as rnd
import p4_aux as p4

In [ ]:
# Ejercicio 1c
np.random.seed(1234)

def shuffle_match(n):
    mazo = np.arange(n)
    perm = list(np.random.permutation(mazo))
    return [ i == e for i, e in enumerate(perm)]

print("Match 1 to 10")
for e in [2,3,4,5,6]:
    nsim = 10**e
    acc = 0
    for _ in range(nsim):
        deck = shuffle_match(100)
        acc += sum(deck[:10]) == 10  # ignores 11:100
    print(f'{nsim:7} | {acc/nsim}')

print("Match only 1 to 10")
for e in [2,3,4,5,6]:
    nsim = 10**e
    acc = 0
    for _ in range(nsim):
        deck = shuffle_match(100)
        acc += sum(deck[:10]) == 10 and sum(deck) == 10 # only matches at :10
    print(f'{nsim:7} | {acc/nsim}')

In [ ]:
def g_fun(x,k):
    nk = 1/k-1
    return (x**nk) / factorial(int(np.floor(nk)))

x = 5
for e in [2,3,4,5,6]:
    nsim = 10**e
    acc = 0
    u = rnd.random()
    for _ in range(nsim):
        acc += g_fun(x,u)
    print(f'{nsim:7} | {acc}')

In [ ]:
from p3_aux import dice_dn

rnd.seed(1234)

res = {}
acc = 0
while len(res) < 11 and acc < 10**6:
    acc += 1 
    r = dice_dn(6) +  dice_dn(6)
    res[r] = 1 + res.get(r, 0)

print(acc, res)
